# 0. Preparations

In [ ]:
import pandas as pd

In [ ]:
data_url = "https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/04-monitoring/data/results-gpt4o-mini.csv"

In [ ]:
data_df = pd.read_csv(data_url)

In [ ]:
sample_data_df = data_df.iloc[:300]

In [ ]:
model_name = "multi-qa-mpnet-base-dot-v1"

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
embedding_model = SentenceTransformer(model_name)

# 1

In [ ]:
answer_llm = sample_data_df.iloc[0].answer_llm

In [ ]:
answer_llm

In [ ]:
embedding_model.encode(answer_llm)[:10]

# 2

In [ ]:
import numpy as np

In [ ]:
evaluations = []
for idx, row in sample_data_df.iterrows():
    answer_orig = row['answer_orig']
    answer_llm = row['answer_llm']

    sim = embedding_model.similarity(
        embedding_model.encode(answer_orig),
        embedding_model.encode(answer_llm)
    )

    evaluations.append(sim)

In [ ]:
np.quantile(evaluations, 0.75)

# 3

In [ ]:
evaluations = []
for idx, row in sample_data_df.iterrows():
    answer_orig, answer_llm = row['answer_orig'], row['answer_llm']

    sim = embedding_model.similarity(
        embedding_model.encode(answer_orig, normalize_embeddings=True),
        embedding_model.encode(answer_llm, normalize_embeddings=True)
    )

    evaluations.append(sim)

In [ ]:
np.quantile(evaluations, 0.75)

# 4

In [ ]:
from rouge import Rouge

In [ ]:
doc = sample_data_df[sample_data_df['document'] == '5170565b'].iloc[0]

In [ ]:
doc

In [ ]:
rouge_scorer = Rouge(metrics=['rouge-1', 'rouge-2', 'rouge-l'])

In [ ]:
scores = rouge_scorer.get_scores(doc['answer_llm'], doc['answer_orig'])[0]

In [ ]:
scores['rouge-1']

# 5

In [ ]:
np.mean([v['f'] for k, v in scores.items()])

# 6

In [ ]:
from collections import defaultdict

In [ ]:
rouge_metrics_names = ['rouge-1', 'rouge-2', 'rouge-l']

In [ ]:
rouge_scorer = Rouge(metrics=rouge_metrics_names)

In [ ]:
ms = defaultdict(list)
for idx, doc in sample_data_df.iterrows():
    answer_orig, answer_llm = doc['answer_orig'], doc['answer_llm']

    scores = rouge_scorer.get_scores(doc['answer_llm'], doc['answer_orig'])[0]

    for k, v in scores.items():
        ms[k].append(v['f'])

In [ ]:
for mn in rouge_metrics_names:
    avg_m = np.mean(ms[mn])

    print(mn, avg_m)